In [ ]:
import os
import shutil
import json
from tqdm import tqdm
import re
import pandas as pd
import csv
import numpy as np
import copy
import chardet
import tiktoken
import argparse
import jsonlines
import zipfile
from collections import Counter
from ebooklib import epub
import ebooklib
from bs4 import BeautifulSoup
import string

#from langchain import PromptTemplate, LLMChain
#from langchain.chat_models import  AzureChatOpenAI
#from langchain_community.chat_models import AzureChatOpenAI

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate 
from langchain_openai import AzureChatOpenAI


#from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number


In [2]:
parser = argparse.ArgumentParser()
### 超参数
### 写到这里
#args = parser.parse_args()
# 创建一个模拟的 parse_args 函数
def parse_args():
    parser = argparse.ArgumentParser()
    
    # 添加你需要的参数
    parser.add_argument('--param1', type=int, default=10, help='An example integer parameter')
    parser.add_argument('--param2', type=str, default='default_value', help='An example string parameter')
    # 模拟命令行参数
    class Args:
        is_series = False
        novel_text_type = "txt"


        language = 1  # 小说文本的语言，0是英语，1是中文
        language = 0  # 小说文本的语言，0是英语，1是中文
        
        gen_language = "中文"   # 生成的语言
        gen_language = "english"   # 生成的语言


        repair_raw_text = True
        key_type = "azure"
        max_token_len = 2000
        chapter_type = 1  # 英文章节，设置为1比较稳妥，不然可能会因为行首的 "I ***" 而误判，但是要求格式比较单一
        # 一般应用于一个小说中有多个章节的小故事，例如神探狄仁杰第一部，里面有三个章节《使团喋血记》、《蓝衫记》、《滴血雄鹰》，只抽取其中一个章节
        extract_chapters_start = 0 # 小说从第几章开始截取，0是从头开始，其他的是具体章节数
        extract_chapters_end = -1 # 小说需要截取多少章节，-1是全书，其他的是具体章节数
        # 抽取断点重连，gpt断连后修改
        extract_begin = 0 # 抽取断点重连，从这个值开始抽取直到最后
        extract_end = -1 # 最少抽取多少章节，-1为抽全篇
        if_chapter_cut = 1 # 1是按章节划分，0是按token划分


        #parser.add_argument('--extract_begin', default=0 , type=int, help='抽取断点重连，从这个值开始抽取直到最后')

    # 这里我们手动创建一个模拟的命令行参数对象
    args = Args()
    
    return args

# 使用模拟的 parse_args 函数
args = parse_args()

# 打印参数值
print(args)
print(args.is_series)

<__main__.parse_args.<locals>.Args object at 0x00000294C2D08FA0>
False


In [ ]:


# 读取epub格式小说
def epub_to_txt(epub_path, txt_path):
    book = epub.read_epub(epub_path)
    text = ''
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            soup = BeautifulSoup(item.get_content(), 'html.parser')
            text += soup.get_text() + '\n\n'
    with open(txt_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)

if args.novel_text_type == "epub":
    if args.is_series:
        series_folder = f'{args.folder_path}/novel/epub/{args.name}'
        output_folder = f'{args.folder_path}/novel/{args.name}_epub2'
        os.makedirs(output_folder, exist_ok=True)
        with tqdm(total=len(os.listdir(series_folder)), desc="Processing .epub files") as pbar:  
            for filename in os.listdir(series_folder):
                if filename.endswith(".epub"):
                    epub_path = os.path.join(series_folder, filename)
                    txt_name = os.path.splitext(filename)[0] + "_epub2.txt"
                    txt_path = os.path.join(output_folder, txt_name)
                    epub_to_txt(epub_path, txt_path)
                if filename.endswith(".txt"):
                    shutil.copy(os.path.join(series_folder, filename),os.path.join(output_folder, filename))
                pbar.update(1)  
    else:
        print("epub 格式转换中...")
        epub_path = f'{args.folder_path}/novel/epub/{args.name}.epub'
        txt_path = f'{args.folder_path}/novel/{args.name}_epub2.txt'
        epub_to_txt(epub_path, txt_path)
        print(f"{args.name}文件 epub 格式转换完成...")

    novel_name = f"{args.name}_epub2"




In [ ]:

# key配置（OpenAI, azure）

if args.key_type == "openai":
    from langchain_openai import ChatOpenAI
    from langchain.llms import OpenAI

    key = 'X'
    #key = 'X'
    key_bytes = key.encode()
    os.environ["OPENAI_API_KEY"] = key_bytes.decode('utf-8')
    #llm = ChatOpenAI(
    llm = OpenAI(
        model_name="gpt-3.5-turbo",
        #model_name="gpt-4-turbo",
        temperature=0
    )
elif  args.key_type == "azure":
    #from openai import AzureOpenAI
    #from openai import AzureChatOpenAI
    os.environ["OPENAI_API_TYPE"] = "azure"
    os.environ["OPENAI_API_VERSION"] = "2024-02-01"
    api_key_num = 2
    if api_key_num == -1:
        os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://iegg-gpt3.openai.azure.com/'
        os.environ["AZURE_OPENAI_API_KEY"] = "X"
    elif api_key_num == 1:
        os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://iegg-gpt-10.openai.azure.com/'
        os.environ["AZURE_OPENAI_API_KEY"] = "X"   
    elif api_key_num == 2:
        os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://iegg-gpt-300.openai.azure.com'
        os.environ["AZURE_OPENAI_API_KEY"] = "X" 

    # 如果模型是 gpt 3.5 gpt 4等，不能用Completions模式, 需要用 Chat模式，否则会报错。
    # 报错信息类似 InvalidRequestError: The completion operation does not work with the specified model, gpt-35-turbo.
    # Chat模式，在langchin上是AzureChatOpenAI，所以不能用AzureOpenAI，否则会报错。
    #from langchain.llms import OpenAI
    #from langchain.llms import AzureOpenAI
    llm_agent = AzureChatOpenAI(
        #deployment_name="iegg-gpt-4-32k",
        #deployment_name="iegg-gpt-4-turbo",
        model_name="gpt-4o",
        temperature= 0.5,
    )
    print(llm_agent)


client=<openai.resources.chat.completions.Completions object at 0x00000294C2D09000> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000294C2D487C0> model_name='gpt-4o' temperature=0.5 openai_api_key=SecretStr('**********') openai_proxy='' azure_endpoint='https://iegg-gpt-300.openai.azure.com' openai_api_version='2024-02-01' openai_api_type='azure'


In [ ]:
# 1 - 新小说抽取
 
# 文件路径
# base_folder = args.folder_path

# 临时文件名

base_folder = "C:/Users/DestinyW/Desktop/storytelling/dataset/"
novel_name = "神探狄仁杰1"
novel_name = "HarryPotter1-HarryPotterAndTheSorcerersStone"

# 小说文件应该在 {base_folder}/novel/"中)
novel_folder = f"{base_folder}/novel_data_history/{novel_name}"       # 处理后的文件夹
save_folder = f"{novel_folder}/{novel_name}_extract"                    # 抽取的文件
save_folder_path =  f"{novel_folder}/reorganized_story_{novel_name}"    # 第二部分用的
save_variables = f'{novel_folder}/middle_variable'                      # 存储一些中间变量


In [75]:

Process_variables = {}

if not os.path.exists(novel_folder):
    os.makedirs(novel_folder)
    print(f"已创建_{novel_name}_文件夹")

#"""
# 创建相应存储文件夹

if not args.is_series:        # 单本小说
    novel_file_path = f'{novel_folder}/{novel_name}.txt'  

    if not os.path.exists(f"{base_folder}/novel/{novel_name}.txt"):
        print(f"注意，{base_folder}/novel/{novel_name}.txt文件不存在!")
    if not os.path.exists(novel_file_path):
        print("复制小说ing...")
        shutil.copy(f"{base_folder}/novel/{novel_name}.txt",novel_file_path)
else:                       # 多部系列小说在一个文件夹下
    novel_file_path = f'{novel_folder}/{novel_name}'  
    if not os.path.exists(f"{base_folder}/novel/{novel_name}"):
        print(f"注意，{base_folder}/novel/{novel_name}文件夹不存在!")
    else:
        if not os.listdir(f"{base_folder}/novel/{novel_name}"):
            print(f"注意，文件夹 {base_folder}/novel/{novel_name} 为空!")
    if not os.path.exists(novel_file_path):
        print("复制文件夹ing...")
        shutil.copytree(f"{base_folder}/novel/{novel_name}",novel_file_path)

if not os.path.exists(save_folder):
  os.makedirs(save_folder)
  print(f"已创建{novel_name}_extract文件夹")
else:
    print('文件夹',save_folder,'已经存在')

已创建_HarryPotter1-HarryPotterAndTheSorcerersStone_文件夹
复制小说ing...
已创建HarryPotter1-HarryPotterAndTheSorcerersStone_extract文件夹


In [76]:


def detect_encoding(filename):
    with open(filename, 'rb') as f:
        raw_data = f.read()
    result = chardet.detect(raw_data)
    return result['encoding']
    
#print("is novel series? ",args.is_series)

def contains_roman_numerals(text):
    pattern = r"\b([IVX]+)\b"  #IVXLCDM
    matches = re.findall(pattern, text, re.IGNORECASE)
    if matches:
        return True
    return False

def has_digit(text):
    for char in text:
        if char.isdigit():
            return True
    return False

def contains_all_english_digits(text):
    english_digits = ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten",
                      "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen",
                      "eighteen", "nineteen", "twenty", "thirty", "forty", "fifty", "sixty", "seventy",
                      "eighty", "ninety", "hundred", "thousand", "million", "billion"]
    text = text.lower()
    if contains_roman_numerals(text): # 罗马数字
      return True
    text = text.replace(" ", "")
    for digit in english_digits:

      if text.find(digit) != -1:
        return True
    if has_digit(text): # 阿拉伯数字
      return True
    return False

def is_chapter_line(line,chapter_type):
    # 英文设置为1比较稳妥，不然可能会因为行首的I ***而误判，但是要求格式比较单一
    if chapter_type == 0:
        line_judge = line.replace(" ", "").replace("　", "").replace(",", "").replace(".", "").replace("'", "").replace("“", "").replace("’", "").replace("”", "").replace(";", "").replace(":", "").replace("?", "").replace("‘", "").replace("—", "").replace("-", "").replace("(", "").replace(")", "").replace("\"", "").replace("*", "").replace("!", "")
        return line.strip().startswith('CHAPTER') or line.strip().startswith('Chapter') or contains_roman_numerals(line_judge) or line.strip().startswith('VOLUME')
    elif chapter_type == 1:
        return line.strip().startswith('CHAPTER') or line.strip().startswith('Chapter') or line.strip().startswith('VOLUME')
    elif chapter_type == 2:
        line_judge = line.replace(" ", "").replace(",", "").replace(".", "").replace("'", "").replace("“", "").replace("’", "").replace("”", "").replace(";", "").replace(":", "").replace("?", "").replace("‘", "").replace("—", "").replace("-", "").replace("(", "").replace(")", "").replace("\"", "").replace("*", "").replace("!", "")
        return contains_roman_numerals(line_judge)
    elif chapter_type == 3:         # 针对的是1./n这种类型
        pattern = r"\d+\."
        return re.fullmatch(pattern, line.strip())
    else:
        # 如果 chapter_type 不是 0、1、2 中的一个值，可能需要根据实际情况进行处理
        raise ValueError("Invalid chapter_type value")


In [77]:

# 切片 - 切章节和chunk


if not args.is_series:   
    # 检测文件编码
    try:
    #if True:
        raw_text = open(novel_file_path, encoding="gbk").read()
        print("default utf-8/gbk")
    except:
        print("not utf-8")
        file_encoding = detect_encoding(novel_file_path)
        print("file_encoding: ",file_encoding)

        # 处理 raw_text
        # 根据检测到的编码来打开文件
        raw_text = open(novel_file_path, encoding=file_encoding).read()
else:
    raw_text = ""
    files_series = os.listdir(novel_file_path)
    files_series.sort()  # 按文件名排序
    num = 0
    with tqdm(total=len(files_series)) as pbar:
        for file_name in files_series:
            file_series_path = os.path.join(novel_file_path, file_name)
            if os.path.isfile(file_series_path):
                num += 1 
                raw_text = raw_text+ f"\n VOLUME {num} - {os.path.splitext(file_name)[0]} \n"

                encoding = detect_encoding(file_series_path)
                with open(file_series_path, 'r', encoding=encoding) as f:
                    raw_text += f.read()
            pbar.update(1)  # 更新进度条
    #print(f"文件夹共{num}本小说")

not utf-8
file_encoding:  utf-8


In [ ]:

# 处理一下txt中出现的换行问题
if args.repair_raw_text  and args.language == 0:
    #print(raw_text)
    # 根据换行符分割成行的列表
    lines = raw_text.splitlines()
    raw_text = ""
    # 逐行处理
    judge_space= 0
    chapter_num = 0


    print(len(lines))

    for i in range(len(lines)):
        if is_chapter_line(lines[i], args.chapter_type):

            raw_text += "\n" + lines[i] + "\n"
            chapter_num += 1
        else:
            if lines[i].strip() == "":
                
                if i<len(lines)-1 and  lines[i+1].strip() == "":
                    
                    continue
                elif judge_space == 1:
                    judge_space = 0
                    continue
                else:
                    raw_text += lines[i] + "\n"
            else:
                #print(i,lines[i])
                if lines[i].strip()[-1].isdigit() or (lines[i].strip()[-1] in string.punctuation):
                    #print(i)
                    raw_text += lines[i] + "\n"
                else:
                    #print(i,lines[i])
                    raw_text += lines[i]
                    if lines[i][-1] !=" ":
                        raw_text += " "
                    if i<len(lines)-2 and lines[i+1].strip() == "":
                        judge_space = 1


        # 对每一行进行处理，例如打印或其他操作
        #print(line.strip())  # 使用 strip() 方法去除每行的前后空白字符
    #print(raw_text )

    print(f"chapter_num: {chapter_num}")
    lines = raw_text.splitlines()
    print(len(lines))

3036
chapter_num: 17
2949


In [ ]:

# 切章节
#chapters = [raw_text]


# 用正则简单看下对话数
def count_dialogues(raw_text):
    # 使用正则表达式匹配任意跨行的对话
    raw_text1 = raw_text.replace('\n','')
    dialogue_pattern = re.compile(r'["\'“‘](.+?)["\'”’]', re.DOTALL)
    # 找到所有匹配的对话
    dialogues = re.findall(dialogue_pattern, raw_text)

    # 返回对话的数量
    return len(dialogues)
# 统计对话数量

dialogue_count = count_dialogues(raw_text)
raw_text_len = len(raw_text.replace(r'\n',''))
#print(raw_text[100:1000])
Process_variables["Total char in novel"] = raw_text_len
Process_variables["Total number of dialogues"] = dialogue_count
print(Process_variables)

# 一些切章节的函数（英文）
# contains_roman_numerals(text)： 判断是否是罗马数字
# has_digit(text)： 是否还有数字
# is_chapter_line(line,line_judge,chapter_type): # 判断不同章节类型



{'Total char in novel': 442821, 'Total number of dialogues': 3947}


In [ ]:

chapters = []
chapter_contents = []
chapter_num_flag = 0

if args.language == 0:          #英语抽章节
    print("language: English")
    chapters_name = []

    i = 0
    Flag_content = 2
    for line in raw_text.split('\n'):
        line = line.replace('\u3000',' ')
        Flag = False
        line_judge = line.replace(" ", "").replace("　", "").replace(",", "").replace(".", "").replace("'", "").replace("“", "").replace("’", "").replace("”", "").replace(";", "").replace(":", "").replace("?", "").replace("‘", "").replace("—", "").replace("-", "").replace("(", "").replace(")", "").replace("\"", "").replace("*", "").replace("!", "")
        #print()
        if is_chapter_line(line, args.chapter_type):
            #print("find")
            i = i+1
            # 遇到章节标题,将之前章节内容添加到结果列表
            head = line.strip()
            #print(head)

            if args.chapter_type ==  3:
                head_num = head[0:-1]
                #print(head_num)
            else:
                head_num = head[7:min(20,len(head))]
                #print(head_num)
            #print(head)
            if (contains_all_english_digits(head_num)) or contains_roman_numerals(line_judge): #or has_digit(head[:11])): #and (head.find("'") == -1):
                
                Flag = True
                if Flag and Flag_content<=1:
                    Flag = False
                Flag_content = 0 # 记录上一行是不是标题，0代表是
                #print(Flag)
                #print(head)
        if line:
            Flag_content += 1
        if Flag:
            
            if chapter_contents:
                chapters.append('\n'.join(chapter_contents))
                chapter_contents = []
                chapter_contents.extend([head, '\n'])
                chapter_num_flag += 1
            # 记录当前章节标题
            chapters_name.append(line)
        else:
            # 累积章节内容
            chapter_contents.append(line)

    # 添加最后一个章节内容
    if chapter_contents:
        chapters.append('\n'.join(chapter_contents))

    Process_variables["chapters_name"] = chapters_name
    Process_variables["len(chapters)"] = len(chapters)
    print(f"len(chapters):{len(chapters)}")
    print(chapter_num_flag)
    #for i in range(len(chapters_name)):
    #    print(chapters_name[i])



# 注意，这里只改了中文切分章节把标题加入的逻辑，英文需要额外修改
    

elif args.language == 1:
    # 中文切章节
    # 我觉得这段代码现在比正则表达式好用， 当然这个代码是针对 第xx章 xxxx 这种格式的
    # 如果有的时候你是 ---第xx章--， 要改一下那个startswith那个 或者把前面的特殊符号给strip掉


    for line in raw_text.split('\n'):
        Flag = False
        if line.strip().startswith('第'):
            # 遇到章节标题,将之前章节内容添加到结果列表

            head = line.strip()
            # print(head)
            #head = head[:min(8,len(head))]
            head = head[:min(30,len(head))]
            if head.find('章',1)>0 or head.find('部',1)>0:
                print(head)
                Flag = True

        if Flag:
            if chapter_contents:
                chapters.append('\n'.join(chapter_contents))
                chapter_contents = []
                chapter_contents.extend([head, '\n'])
                
            # 记录当前章节标题
            # chapters.append(line)
        else:
            # 累积章节内容
            chapter_contents.append(line)

    # 添加最后一个章节内容
    if chapter_contents:
        chapters.append('\n'.join(chapter_contents))

    print(len(chapters))

else:
    print("warning! Currently not supported for other languages")

language: English
len(chapters):18
17


In [81]:

# 切chunk


enc = tiktoken.get_encoding("cl100k_base")

# 定义divide函数，用来切分超长文本
def divide_str(s, sep=['\n', '.', '。']):
    mid_len = len(s) // 2  # 中心点位置
    best_sep_pos = len(s) + 1  # 最接近中心点的分隔符位置
    best_sep = None  # 最接近中心点的分隔符
    for curr_sep in sep:
        sep_pos = s.rfind(curr_sep, 0, mid_len)  # 从中心点往左找分隔符
        if sep_pos > 0 and abs(sep_pos - mid_len) < abs(best_sep_pos - mid_len):
            best_sep_pos = sep_pos
            best_sep = curr_sep
    if not best_sep:  # 没有找到分隔符
        return s, ''
    return s[:best_sep_pos + 1], s[best_sep_pos + 1:]
def strong_divide(s):
    left, right = divide_str(s)

    if right != '':
        return left, right

    whole_sep = ['\n', '.', '，', '、', ';', ',', '；',\
                 '：', '！', '？', '(', ')', '”', '“', \
                 '’', '‘', '[', ']', '{', '}', '<', '>', \
                 '/', '''\''', '|', '-', '=', '+', '*', '%', \
               '$', '''#''', '@', '&', '^', '_', '`', '~',\
                 '·', '…']
    left, right = divide_str(s, sep=whole_sep)

    if right != '':
        return left, right

    mid_len = len(s) // 2
    return s[:mid_len], s[mid_len:]

# 以1500 token为限，切分chunk，输出总chunk数量
# max_token_len = 24000 #基本是一章，太大了
max_token_len = args.max_token_len   # 对话抽取5000效果可以,但是有点犯病
chunk_text = []

chapter_num = -1
extract_chapters_start = args.extract_chapters_start
extract_chapters_end = args.extract_chapters_end



# 这里要注释掉
# 这里是你要截取多少章节，比如说一部小说有3个故事，其中，想要第二个故事，则设置章节数开头和结尾
# 选取所有就设置为-1或者注释（默认是-1）
extract_chapters_end = 17
#extract_chapters_start = 2



print("章节数：",len(chapters))
chunk_text = []
chapter_text_token = []

for chapter in chapters:
    

    #chapter_text_token = []
    # 如果截取章节从0开始，0一般是第一章前的介绍或者序章
    if (not extract_chapters_end == -1) and chapter_num >= extract_chapters_end:
        print(f"截取章节数：{extract_chapters_start} - {extract_chapters_end}")
        break
    
    chapter_num += 1
    #print(chapter_num, extract_chapters_start)
    if chapter_num < extract_chapters_start:
        continue

    # 找到第一个换行符的位置
    newline_index = chapter.find('\n')


    # 如果找到了换行符，就取从开始到换行符之前的部分，否则取前30个字符
    print(chapter[:min(30, newline_index) if newline_index != -1 else 30])

    split_text = chapter.split('\n')
    tmp = []

    for line in split_text:
        # 有无影响？
        if not line.strip():
            continue

        line_len = len(enc.encode( line ))

        if line_len <= max_token_len - 5:
            tmp.append(line)
        else:
            # print('divide line with length = ', line_len)
            path = [line]
            tmp_res = []

            while path:
                my_str = path.pop()
                left, right = strong_divide(my_str)

                len_left = len(enc.encode( left ))
                len_right = len(enc.encode( right ))

                if len_left > max_token_len - 15:
                    path.append(left)
                else:
                    tmp_res.append(left)

                if len_right > max_token_len - 15:
                    path.append(right)
                else:
                    tmp_res.append(right)

            for line in tmp_res:
                tmp.append(line)

    split_text = tmp

    #print(split_text)
    #if chapter_num >2:
    #    break


    curr_len = 0
    curr_chunk = ''
    chunk_text_per_chapter = []
    for line in split_text:
        line_len = len(enc.encode( line ))

        if line_len > max_token_len:
            print('warning line_len = ', line_len)

        if curr_len + line_len <= max_token_len:
            curr_chunk += line + '\n'
            curr_len += line_len + 1
        else:
            chunk_text_per_chapter.append(curr_chunk)
            curr_chunk = line
            curr_len = line_len

    if curr_chunk:
        chunk_text_per_chapter.append(curr_chunk)

    chunk_text.extend(chunk_text_per_chapter) 

    #print(len(chunk_text))
    # break
    if args.if_chapter_cut == 1:
        chapter_text_token.append(chunk_text_per_chapter)
        #print("chapter与len",chapter_num,len(chapter_text_token))   # 如果是从头开始计算，chapter_num 应该比 len（chapter）少1
        print("chapter_num: ",chapter_num,"-",len(chapter_text_token[-1]),"-",len(chunk_text))
        #chunk_text = []
    #print(len(chapter_text_token))
    #print(len(chapter_text_token[-1]))

Process_variables["max_token_len"] = max_token_len
Process_variables["分块后块数"] = len(chunk_text)

print(f"分块后块数: {len(chunk_text)}")

章节数： 18
1.Harry Potter and the Sorcere
chapter_num:  0 - 1 - 1
CHAPTER ONE THE BOY WHO LIVED
chapter_num:  1 - 4 - 5
CHAPTER TWO
chapter_num:  2 - 3 - 8
CHAPTER THREE
chapter_num:  3 - 3 - 11
CHAPTER FOUR
chapter_num:  4 - 3 - 14
CHAPTER FIVE
chapter_num:  5 - 5 - 19
CHAPTER SIX
chapter_num:  6 - 5 - 24
CHAPTER SEVEN
chapter_num:  7 - 4 - 28
CHAPTER EIGHT
chapter_num:  8 - 3 - 31
CHAPTER NINE
chapter_num:  9 - 4 - 35
CHAPTER TEN
chapter_num:  10 - 4 - 39
CHAPTER ELEVEN
chapter_num:  11 - 3 - 42
CHAPTER TWELVE
chapter_num:  12 - 4 - 46
CHAPTER THIRTEEN
chapter_num:  13 - 3 - 49
CHAPTER FOURTEEN
chapter_num:  14 - 3 - 52
CHAPTER FIFTEEN
chapter_num:  15 - 4 - 56
CHAPTER SIXTEEN
chapter_num:  16 - 5 - 61
CHAPTER SEVENTEEN
chapter_num:  17 - 4 - 65
分块后块数: 65


In [82]:
prompt=PromptTemplate(
        template="请问，{country}的首都是哪里 ?",
        input_variables=["country"],
    )

chain = prompt | llm_agent
a = ""
#print(chain.invoke("日语"))
#a = chain.invoke("日语")
print(a)
#print(a.response_metadata["token_usage"])

In [83]:
# 剧情摘要


system_prompt_0 = """
Summarize the key points of the following text in a concise way, using bullet points.
"""
q_example_0 = """###
Text:
洪七公、周伯通、郭靖、黄蓉四人乘了小船，向西驶往陆地。黄蓉不住向周伯通详问骑鲨游海之事，周伯通兴起，当场就要设法捕捉鲨鱼，与黄蓉大玩一场。
洪七公被欧阳锋点中之后，穴道虽已解开，内伤却又加深了一层。黄蓉喂他服了几颗九花玉露丸，痛楚稍减，气喘仍是甚急。
老顽童不顾别人死活，仍是嚷着要下海捉鱼，黄蓉向他连使眼色，要他安安静静的，别吵得洪七公心烦。周伯通并不理会，只闹个不休。黄蓉皱眉道：“你要捉鲨鱼，又没饵引得鱼来，吵些甚么”

Summarize in BULLET POINTS form:
"""
a_example_0 = """
- 洪七公等四人乘船西行,洪七公因受内伤加重而气喘不止
- 周伯通要捉鲨鱼玩,被黄蓉阻止
"""
messages = [SystemMessage( content = system_prompt_0),
            HumanMessage( content = q_example_0),
            AIMessage( content = a_example_0)]

#"""

print("save_folder: ",save_folder)
print("len(chunk_text): ",len(chunk_text))
print("chunk_text[1][:200]: ",chunk_text[1][:20])
#print("chain: ",chain)

save_folder:  C:/Users/DestinyW/Desktop/storytelling/dataset//novel_data_history/HarryPotter1-HarryPotterAndTheSorcerersStone/HarryPotter1-HarryPotterAndTheSorcerersStone_extract
len(chunk_text):  65
chunk_text[1][:200]:  CHAPTER ONE THE BOY 


In [84]:
print(chunk_text[0])
print(len(chunk_text[0]))

1.Harry Potter and the Sorcerer's Stone.txt Harry Potter and the Sorcerer's Stone 

83


In [ ]:
# 按token
# 和下面的二选一
# 按章节可能过长，时间异常长
# 尽量按选章节
if False:
    args.extract_end = -1
    # 最后一个章节数+1
    args.extract_end = 10
    # 全篇
    if args.extract_end <= 0:
        extract_part = len(chunk_text)  # 全篇
    else :
        extract_part = args.extract_end

    print(len(chunk_text))

    for i in tqdm(range(args.extract_begin, min(len(chunk_text),extract_part))):  #断点重连
        
        save_name_sum = os.path.join(save_folder, f"{i}_sum.txt")
        # 文本过短，跳过
        if len(chunk_text[i])<30:
            continue

        if not os.path.exists(save_name_sum) or os.path.getsize(save_name_sum) < 5:
            if os.path.exists(save_name_sum):
                print('re-summarize id = ',i )
            #dealing with summarize
            messages = [SystemMessage( content = system_prompt_0),
                HumanMessage( content = q_example_0),
                AIMessage( content = a_example_0)]

            new_input = f"###\nText:\n{chunk_text[ i ]}\nSummarize in BULLET POINTS form:"

            messages.append( HumanMessage(content = new_input) )

            summarize_response = llm_agent.invoke(messages).content

            with open(save_name_sum, 'w', encoding='utf-8') as f:
                f.write( summarize_response )

        raw_text_save_name = os.path.join(save_folder, f"{i}_raw.txt")
        if not os.path.exists(raw_text_save_name) or os.path.getsize(raw_text_save_name) < 5:
            with open(raw_text_save_name, 'w', encoding='utf-8') as f:
                f.write( chunk_text[i] )
        #break

else:
    # 按章节
    args.extract_end = -1
    #args.extract_end = 10
    # 全篇
    if args.extract_end <= 0:
        extract_part = len(chapter_text_token)  # 全篇
    else :
        extract_part = args.extract_end


    print(len(chapter_text_token))

    for i in tqdm(range(args.extract_begin, min(len(chapter_text_token),extract_part))):  #断点重连

        save_name_sum = os.path.join(save_folder, f"{i}_sum.txt")

        query_text_sum = ""
        summary_text_sum = ""
        for chapter_cut_text in chapter_text_token[i]:

            #print(len(chapter_cut_text))
            

            query_text_sum += chapter_cut_text
            messages = [SystemMessage( content = system_prompt_0),
                HumanMessage( content = q_example_0),
                AIMessage( content = a_example_0)]
            new_input = f"###\nText:\n{chapter_cut_text}\nSummarize in BULLET POINTS form:"
            messages.append( HumanMessage(content = new_input) )
            summarize_response = llm_agent.invoke(messages).content
            summary_text_sum += summarize_response
        # 文本过短，跳过
        if len(query_text_sum)<30:
            continue
            
        with open(save_name_sum, 'w', encoding='utf-8') as f:
            f.write( summary_text_sum )

        raw_text_save_name = os.path.join(save_folder, f"{i}_raw.txt")
        if not os.path.exists(raw_text_save_name) or os.path.getsize(raw_text_save_name) < 5:
            with open(raw_text_save_name, 'w', encoding='utf-8') as f:
                f.write( query_text_sum )

        #break



18


100%|██████████| 18/18 [15:25<00:00, 51.41s/it]


In [86]:
print(json.dumps(chapter_text_token,indent=2,ensure_ascii=False))

[
  [
    "1.Harry Potter and the Sorcerer's Stone.txt Harry Potter and the Sorcerer's Stone \n"
  ],
  [
    "CHAPTER ONE THE BOY WHO LIVED\nMr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense.\n  Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors. The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere.\n  The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it. They

In [87]:
def split_string_by_lines(input_string, max_token_len):
# 按行与token切分

    cut_proportion = 2
    # 初始化结果列表
    str_result = []

    input_token = len(enc.encode(input_string))
    if input_token < max_token_len:
    #if len(input_string) < max_token_len:    
        str_result = [input_string]
        print(f"input char = {len(input_string)}, input token = {input_token}")
    else:
        len_str_ori = len(enc.encode(input_string))
        #len_str_ori = len(input_string)

        lines = input_string.split('\n')

        #for line in lines:
        #    print(line)
            
        str_result = [""]
        i = 0 
        if len_str_ori > max_token_len*cut_proportion:
            while i<len(lines):

                line_len = len(enc.encode( lines[i] ))  # token长度，可以换成字符长度
                if (len(enc.encode(str_result[-1]))+ line_len) < max_token_len:
                #if (len(str_result[-1])+ len(lines[i])) < max_token_len: 
                #    
                    str_result[-1] += lines[i] + '\n'
                else:

                    if len_str_ori <= max_token_len*cut_proportion:
                        break
                    str_result.append(lines[i])
                len_str_ori -= len(enc.encode(lines[i]))
                len_str_ori -= len(lines[i])
                i += 1
            
        #print("len1:", len_str_ori)
        len_str_ori /= 2
        str_result.append("")
        
        while (len(enc.encode(str_result[-1]))+len(enc.encode( lines[i] )) < len_str_ori):
        #while (len(str_result[-1])+len(lines[i]) < len_str_ori):
            str_result[-1] += lines[i] + '\n'
            i += 1
        str_result.append("")
        #print(str_result)
        
        for j in range(i,len(lines)):
            str_result[-1] += lines[j] + '\n'
    for i in range(len(str_result)):
        str_result[i] = str_result[i].strip('\n')

    cleaned_list = [item for item in str_result if item]
    return cleaned_list

In [88]:
system_prompt_1 = """
Summarize the above key points into a short paragraph, as short as possible
"""
q_example_1 = """###
Text:
- 郡主企图刺杀狄公，虎敬晖牺牲自己保护狄公。
- 郡主揭露自己是真正的金木兰，旨在推翻武则天的统治。
- 郡主批评太子软弱，表明自己利用外力对抗内敌的策略。
- 狄公和郡主就忠诚和策略发生争执，郡主强调她的目的是推翻武则天。
- 郡主努力三年联络各种势力营救刘金，曾策划失败的营救行动
- 武则天的圣旨让郡主嫁给突厥可汗，郡主视为救刘金及自身脱离牢笼的机会
- 郡主与吉利可汗的叔叔莫度密谋，计划在团进京时假死逃脱
- 郡主自认为是李青霞，狄仁杰因此未能识破其真实身份
- 狄仁杰愤怒郡主勾结突厥，郡主则声称要成为女皇帝
- 虎敬晖为保护狄仁杰而攻击郡主，最后牺牲
Summarize in BULLET POINTS form:
"""
a_example_1 = """
郡主企图刺杀狄公，虎敬晖因此牺牲。她自称金木兰，计划利用外力推翻武则天，成为女皇帝。之前营救刘金失败，此次武则天令她嫁给突厥可汗，她借此机会与可汗叔叔密谋假死逃脱。狄仁杰虽未识破她的真实身份，但对她勾结外敌表示愤怒。
"""

prompt_resum=PromptTemplate(
    template = """接下来我将给你一段摘要，你需要将这些摘要再次总结精炼成一段叙述性的话
        用{language}回答,不要提到"好的","明白了"这种令人出戏，没有意义的字眼
        原始摘要如下：{long_summary}

    """,
    input_variables=["language","long_summary"],
)
# 你需要找出其中最重要的至多6个人物进行介绍

In [89]:
import os
# 对总结内容重新继续总结
# 初始化一个列表来存储文件内容
max_token_len
# 遍历文件夹中的文件
k_num = 0
for filename in sorted(os.listdir(save_folder)):
    
    # 检查文件是否以 '_sum.txt' 结尾
    if filename.endswith('_sum.txt'):
        k_num += 1
        file_path = os.path.join(save_folder, filename)
        # 打开并读取文件
        print(filename,f" {k_num}/{len(chapter_text_token)}")
        with open(file_path, 'r', encoding='utf-8') as file:
            content_plot = file.read()  # 读取文件内容
        con_list = split_string_by_lines(content_plot,max_token_len)
        #print(json.dumps(con_list,indent = 2,ensure_ascii=False))
        #break
        print(f"该段文件总长度：{len(content_plot)}, 分为{len(con_list)}段")
        plot_text_sum = ""
        #"""
        for list_1 in con_list:
            print(len(list_1))    # 输出每段文件长度
            if True:
                chain_role_list = prompt_resum | llm_agent
                summarize_response_1 = chain_role_list.invoke({"language":args.gen_language,"long_summary": list_1}).content
                print(summarize_response_1)
                #break
            else:
                messages_1 = [SystemMessage( content = system_prompt_1),
                    HumanMessage( content = q_example_1),
                    AIMessage( content = a_example_1)]
                new_input_1 = f"###\nText:\n{list_1}\nSummarize in BULLET POINTS form:"
                messages_1.append( HumanMessage(content = new_input_1) )
                summarize_response_1 = llm_agent.invoke(messages_1).content
            plot_text_sum += summarize_response_1 +'\n'
        #print(plot_text_sum)
        #break
        file_path_plot = file_path.replace("_sum.txt","_resum.txt")
        with open(file_path_plot, 'w', encoding='utf-8') as f:
            f.write( plot_text_sum )
        #break
        #"""

0_sum.txt  1/18
input char = 116, input token = 29
该段文件总长度：116, 分为1段
116
A text file titled "Harry Potter and the Sorcerer's Stone.txt" contains the booked for the event. The company is also expected to make a major announcement about the future of the Unreal Engine. Unreal Engine 4 was announced back in December 2011, but was not shown to the public until May of last year.

The Game Developers Conference is set to take place March 25-29 in San Francisco. We'll be sure to keep you updated on any announcements as they happen.

Our Take
Epic Games has always been on the cutting edge of technology, and we can't wait to see what the company has been working on. The fact that it will be showing off its new technology at GDC, which is historically. So I think there's a lot of good things going on. - And lastly, if you could just talk about the importance of the conference, just getting people together to talk about the issues. - Well, I think for one thing, you know, the conference is import

In [90]:
# 将总结文件和原始文本标题(原文本第一行)结合

for filename in sorted(os.listdir(save_folder)):
    # 检查文件是否以 '_plot.txt'或者"resum.txt" 结尾
    if filename.endswith('_resum.txt'):
        content_plot_sum = ""
        #print("a")
        novel_plot_path = os.path.join(save_folder, filename)
        novel_raw_path = novel_plot_path.replace("_resum.txt","_raw.txt")
        with open(novel_plot_path, 'r', encoding='utf-8') as file:
            content_plot = file.read()  # 读取重新总结文件内容
        with open(novel_raw_path, 'r', encoding='utf-8') as file:
            content_raw = file.read()  # 读取源文件章节

        content_raw_list = content_raw.split('\n')
        i = 0
        while i < len(content_raw_list) and (not content_raw_list[i].strip()):
            i += 1
        #print( content_raw_list[i])
        content_plot_sum += content_raw_list[i]
        content_plot_sum += '\n\n'

        content_plot_sum += content_plot.strip('\n')
        content_plot_sum += '\n\n\n'

        content_plot_sum = content_plot_sum.strip('\n')
        #print(content_plot_sum)
        with open(novel_plot_path, 'w', encoding='utf-8') as f:
            f.write( content_plot_sum )

这里已经是天然的单线剧情了, 重整剧情, 后面直接参照game抽取流程抽取背景和单线剧情

In [110]:
# 这里已经是天然的单线剧情了, 重整剧情, 后面直接参照game抽取流程抽取背景和单线剧情

#a = llm_agent.invoke(f"你知道小说{novel_name}吗，请详细解释他的主要内容和主要人物").content
#print(a)

# 这里希望整合到extra_game中


novel_plot_save_dict = {}
for filename in sorted(os.listdir(save_folder)):
    # 检查文件是否以 '_plot.txt'或者"resum.txt" 结尾
    if filename.endswith('_resum.txt'):
        #print("a")
        novel_plot_path = os.path.join(save_folder, filename)
        with open(novel_plot_path, 'r', encoding='utf-8') as file:
            content_plot = file.read()  # 读取重新总结文件内容
        #print(content_plot)
        plot_num = int(filename.replace('_resum.txt',""))
        #print(plot_num)
        novel_plot_save_dict[plot_num] = content_plot

# 补充信息，比如是小说的第一部分
novel_name_supple = ""
#novel_name_supple = ": 第一部 使团喋血记"
novel_save_dict = {"name":novel_name + novel_name_supple,"chapter":[],"plot":novel_plot_save_dict,"type":"novel"}
print(json.dumps(novel_save_dict,indent=2,ensure_ascii=False))
save_dict_folder =  "C:/Users/DestinyW/Desktop/storytelling/extra_plot/data/novel_plot_dict.jsonl"
with open(save_dict_folder, 'a', encoding='utf-8') as f:  # 使用'a'模式以追加方式打开文件
    json.dump(novel_save_dict, f,ensure_ascii=False)  # 将字典转换为JSON格式并写入文件
    f.write('\n')  # 在每个字典后面添加换行符，确保每个字典占一行

{
  "name": "HarryPotter1-HarryPotterAndTheSorcerersStone: 第一部 使团喋血记",
  "chapter": [],
  "plot": {
    "0": "1.Harry Potter and the Sorcerer's Stone.txt Harry Potter and the Sorcerer's Stone \n\nA text file titled \"Harry Potter and the Sorcerer's Stone.txt\" contains the booked for the event. The company is also expected to make a major announcement about the future of the Unreal Engine. Unreal Engine 4 was announced back in December 2011, but was not shown to the public until May of last year.\n\nThe Game Developers Conference is set to take place March 25-29 in San Francisco. We'll be sure to keep you updated on any announcements as they happen.\n\nOur Take\nEpic Games has always been on the cutting edge of technology, and we can't wait to see what the company has been working on. The fact that it will be showing off its new technology at GDC, which is historically. So I think there's a lot of good things going on. - And lastly, if you could just talk about the importance of the co

### 中断之后从这里开始

In [13]:
# 反向读取novel_save_dict
save_dict_folder = "C:/Users/DestinyW/Desktop/storytelling/extra_plot/data/novel_plot_dict.jsonl"
# 读取.jsonl文件并解析每一行的JSON数据

novel_save_list = []
with open(save_dict_folder, 'r', encoding='utf-8') as f:  # 使用'r'模式以读取方式打开文件，并指定编码为utf-8
    for line in f:
        novel_save_list.append(json.loads(line))# 将每一行的JSON数据解析为Python字典


# 这里应该是一个大循环
novel_save_dict = novel_save_list[-1]
print(json.dumps(novel_save_dict, indent=2, ensure_ascii=False))  # 打印解析后的字典，美化输出

{
  "name": "HarryPotter1-HarryPotterAndTheSorcerersStone",
  "chapter": [],
  "plot": {
    "10": "CHAPTER TEN\n\nHarry and Ron, excited about their adventure with a three-headed dog and curious about a mysterious package moved from Gringotts to Hogwarts, receive a Nimbus Two Thousand broomstick, making Malfoy envious. Despite Malfoy's attempts to get Harry in trouble, Professor Flitwick supports Harry. After learning Quidditch basics from Oliver Wood and practicing, Harry gets busy with training and schoolwork. On Halloween, after an argument with Ron, Hermione skips class but is later found in danger from a troll in the dungeons. Harry and Ron bravely rescue her, leading to a scolding from Professor McGonagall but earning points for bravery. Hermione lies to protect them, and the shared experience of defeating the troll solidifies their friendship.",
    "11": "CHAPTER ELEVEN\n\nIn November, as cold weather set in at Hogwarts, Harry prepared for his first Quidditch match against Sly

### 这里是extra_game的代码

In [ ]:
# 可配置选项


data_folder = "./data"
# 合并粒度(推荐1-4，越小划分的越细,每n章合并,这个参数为0时,代表全文合并；参数为1时，代表保持原主线结构不变)
# 这个参数为0时,代表全文合并,这个参数需要小于seg_main_plot_num

combine_plot_num = 1
combine_plot_num = 0
# 小说划分粒度,主线划分为多少章节(推荐6-10)
seg_main_plot_num = 10
#seg_main_plot_num = 6
# 语言(0时中文，1是english，只接受这两个参数)
# 这里要改一下
use_language = args.gen_language
language = use_language
# 多线剧情发散度（生成结局数量，等？）(生成范围/+1)
branch_num = 2
#role_num = 3

# 生成多线还是单线剧情
# true是单线
if_single_plot = True
#if_single_plot = False

# 是否使用小说文本
#if_use_novel_doc = False
#if_use_novel_doc = True
# 小说中是否具有额外信息还是需要全部自己抽取
#if_novel_extra_info = False


gen_parameter_dict = {"seg_main_episode_num":seg_main_plot_num,"combine_episode_num":combine_plot_num,"branch_num":branch_num,"language":use_language,"if_single_episode":if_single_plot}



if combine_plot_num>seg_main_plot_num or combine_plot_num<0:
    print(f"warning! wrong parameter for combine_plot_num {combine_plot_num}")


# 键值对模板
# story，plot，messages
template_dict_story = {
    "Settings":{
        "story_id": "",         # 唯一id，用于查找
        "story_name": "",       # 故事名
        "story_style": "",      # 故事风格，尽可能超过5个短语
        "leading_role":"",      # 每个plot中都要出现主角
        "story_desc": "",       # 故事的主要描述
        "story_goal":"",      # 故事主角需要做的目标
        "story_chars": {},      # dict, 故事有哪些主角,性格以及人物描述 
        "storylines":[],      # list, 故事主线梗概
        "states":[],            # 由多个list组成, 每个list代表这一个剧情顺序
        "story_type":"",        # 故事类型，从哪里获得的，game/novel...
        "story_struct":"",         # 故事结构，单线/多线
        "story_language":"",          # 故事语言，中文/英文
        "gen_parameter":{}      # 生成故事使用的超参数
    },
    "Episodes":[]   # plots节点, plot 是list，多个下面的plot
}
template_dict_plot = {
    "episode_id": "",
    "pre_episode_id":[],
    "episode_goal": "",
    "episode_scene": "",
    "episode_desc": "",
    "episode_chars": {},
    "extra_info": [],
    "triggers": [
#        {
#            "condition": "",
#            "examples": [],
#            "next_episode": "",
#        },
    ],
    "message_pairs":[],
}
template_dict_message =  {
    "task": ["transition", "role-play", ""],
    "user": "@npc content_text",
    "bot": {
        "narrative": "", 
        "role_dialogue": [
            {"name": "", 
             "utterance": "",
             },
        ],
        "next_episode": ""
    }                       
}



# 抽取剧情相关信息
prompt_background=PromptTemplate(
    template = """请介绍在{novel_name}小说中以下内容：
        小说故事风格(使用几个短语形容，使用逗号隔开,7个词以上)，发生的背景以及剧情的主要目标和主角分别是什么
        介绍时不要出现“读者”，“小说”等令人出戏的单词
        保持结果的格式和例子中的格式一致, 并以键值对的json格式: '{{key:value, ...}} '重构。
        用{language}回答,下面是一个例子
        {{
        "story_style":"奇幻,中世纪，成人向叙事,复杂阴谋,黑暗幻想,史诗冒险,传奇主义色彩,哥特式神秘感,魔法与剑"
        "background":"这是一个辽阔的幻想世界，这个世界充满了魔法、怪兽和战争，猎魔人杰洛特是这个世界中的一名传奇人物，他的养女希里拥有强大的魔法能力，被来自另一个维度，拥有强大的力量且神秘的狂猎势力追捕。杰洛特为了寻找并保护希里，踏上了一段跨越战争蹂躏的大陆的冒险旅程，同时卷入了北方王国与尼尔夫加德帝国之间的冲突。猎魔人是被训练来对抗怪物的变种人类，尽管他们拥有超自然力量，却常被社会边缘化。"
        "target":"杰洛特的主要目标是是寻找并保护他的养女希里，她是一个拥有强大力量的源术士，目前正在被神秘的狂猎骑士团追捕，同时杰洛特还需要揭露狂猎的计划，最终阻止他们。在这个过程中需要探索一个充满危险和奇幻元素的广阔开放世界，解决各种任务和委托，与怪物战斗，同时做出影响剧情走向的决策。杰洛特还需要不断提升自己的能力，包括剑术、魔法和装备，以应对越来越强大的挑战。"
        "leading_role":"杰洛特"
        }}
        小说摘要如下: {plot_summary}
    """,
    input_variables=["novel_name","language","plot_summary"],
)




In [ ]:
print

In [ ]:
# 背景 等内容dict填充


novel_save_dict = novel_save_list[-1]

novel_name = novel_save_dict["name"]
novel_plot_summary = novel_save_dict["plot"]

res_dic = {}
print(f"小说名称：{novel_name}")

chain0 = prompt_background | llm_agent
res0 = chain0.invoke({"novel_name":novel_name, "language":language, "plot_summary":novel_plot_summary}).content
try:
    cleaned_json_string0 = res0.split('```json')[1].split('```')[0].strip()
except:
    cleaned_json_string0 = res0

res0_dict = {}

res0_dict = json.loads(cleaned_json_string0)    
#print(res0_dict)

if type(res0_dict) == list:
    for item in res0_dict:
        item1 = item.split(":")
        res0_dict[item1[0].strip()] = item1[1].strip()


story_dict = copy.deepcopy(template_dict_story)

story_dict["Settings"]["story_name"] = novel_name
style_items = re.split(r'[，,]', res0_dict["story_style"])
story_dict["Settings"]['story_style'] = [item.strip() for item in style_items if item.strip()]

# 故事介绍是是背景还是背景加目标？
story_dict["Settings"]['story_desc'] = res0_dict["background"]
story_dict["Settings"]["story_goal"] = res0_dict["target"]
#story_dict["story_info"]['story_desc'] = res0_dict["background"] + '/n' + res0_dict["target"]

story_dict["Settings"]["leading_role"] = res0_dict["leading_role"]
story_leading_role = res0_dict["leading_role"]
story_dict["Settings"]["story_type"] = novel_save_dict["type"]
story_dict["Settings"]["story_struct"] = "single" if if_single_plot else "multi"
story_dict["Settings"]["story_language"] = language
story_dict["Settings"]["gen_parameter"] = gen_parameter_dict


# 这里存疑：第一行是否是章节
if True:
    for i in range(len(novel_plot_summary)):
        plot_i = novel_plot_summary[str(i+1)].split("\n",1)
        #print(novel_plot_summary[i])
        #print(plot_i)
        story_dict["Settings"]["storylines"].append(plot_i[0])

print(json.dumps(story_dict,indent=2, ensure_ascii=False))


小说名称：HarryPotter1-HarryPotterAndTheSorcerersStone
{
  "Settings": {
    "story_id": "",
    "story_name": "HarryPotter1-HarryPotterAndTheSorcerersStone",
    "story_style": [
      "magical adventure",
      "coming-of-age",
      "whimsical",
      "mysterious",
      "heartwarming",
      "humorous",
      "fantastical"
    ],
    "leading_role": "Harry Potter",
    "story_desc": "Set in the magical world of Hogwarts School of Witchcraft and Wizardry, as well as various locations in the non-magical world, the story follows Harry Potter, an orphan who discovers on his eleventh birthday that he is a wizard. He is introduced to the magical world, where he learns about his parents' legacy, his own magical abilities, and the dark wizard Voldemort who killed his parents. The school is filled with enchanted creatures, magical spells, and hidden secrets, creating an atmosphere of wonder and discovery.",
    "story_target": "The main goal is for Harry to uncover the mystery of the Sorcerer's 

In [16]:
print(i)

16


### 目前只有单线（多线在思考）

如果是多线，这里应该有一个多线重组织

In [17]:
print(json.dumps(novel_plot_summary,indent=2, ensure_ascii=False))

{
  "10": "CHAPTER TEN\n\nHarry and Ron, excited about their adventure with a three-headed dog and curious about a mysterious package moved from Gringotts to Hogwarts, receive a Nimbus Two Thousand broomstick, making Malfoy envious. Despite Malfoy's attempts to get Harry in trouble, Professor Flitwick supports Harry. After learning Quidditch basics from Oliver Wood and practicing, Harry gets busy with training and schoolwork. On Halloween, after an argument with Ron, Hermione skips class but is later found in danger from a troll in the dungeons. Harry and Ron bravely rescue her, leading to a scolding from Professor McGonagall but earning points for bravery. Hermione lies to protect them, and the shared experience of defeating the troll solidifies their friendship.",
  "11": "CHAPTER ELEVEN\n\nIn November, as cold weather set in at Hogwarts, Harry prepared for his first Quidditch match against Slytherin, with his role as Seeker initially kept secret. Hermione helped him with homework an

In [ ]:
if if_single_plot:
    pre_plot_dict = {}
    single_plot_dict = {}
    pre_plot_dict[1] = 0
    for i in range(len(story_dict["Settings"]["storylines"])-1):
        single_plot_dict[i+1] = {"plot_content":novel_plot_summary[str(i+1)],"next_episode":[{i+2:novel_plot_summary[str(i+2)]}],"info": "plot"}
        pre_plot_dict[i+2] = [i+1]
        #print(main_plot_list["main"][i])
    novel_plot_summary[str(i+1)].split("\n",1)
    # 补充结局节点
    single_plot_dict[len(novel_plot_summary)] = {"plot_content":novel_plot_summary[str(len(novel_plot_summary))],"next_episode":[],"info": "end"}
    #pre_plot_dict[len(novel_plot_summary)] = [len(novel_plot_summary)-1]
    # 顺序记录所有节点
    story_dict["Settings"]["states"] = [[i+1 for i in range(len(story_dict["Settings"]["storylines"]))]]

    print(json.dumps(single_plot_dict,indent=2, ensure_ascii=False))  


print("剧情树: ",story_dict["Settings"]["states"])
print("前一个节点的dict: ",pre_plot_dict)


{
  "1": {
    "plot_content": "CHAPTER ONE THE BOY WHO LIVED\n\nMr. and Mrs. Dursley of Privet Drive, who took pride in their normalcy, were unsettled by strange occurrences and mentions of the Potters, especially their son Harry. Meanwhile, Albus Dumbledore and Professor McGonagall discussed the defeat of Voldemort and the Potters' deaths. Dumbledore decided to leave baby Harry with the Dursleys, believing he would be safer away from the magical world's fame. Hagrid delivered Harry on a flying motorcycle, and after leaving a letter, Dumbledore wished Harry good luck and disappeared, as the magical community secretly celebrated Harry as \"the boy who lived.\"",
    "next_plot": [
      {
        "2": "CHAPTER TWO\n\nHarry Potter has spent nearly ten miserable years living in a cupboard under the stairs at the Dursleys' home, where he is treated poorly by his relatives. On Dudley's birthday, after a series of events including Mrs. Figg breaking her leg, the Dursleys reluctantly take Ha

### 生成对应节点

In [ ]:

# 2. 其他信息
prompt_plot_info=PromptTemplate(
    template = """请介绍在{novel_name}小说中的某段剧情里以下内容，用{language}回答,：
        1.target: 在这一个plot中主要目标是什么，应该以主角:{story_leading_role}作为主体进行叙述
        2.scene: 这段情节的背景和它发生的地点，需要结合当前plot中的剧情进行介绍
        3.role: 这一段plot中出现的主要人物，需要介绍性格，在这一幕的状态等信息，主要人物数量不要超过5个

        使用如下格式`**:详细内容`, 保持结果的格式和例子中的格式一致, 并以json格式: '{{key:value}}(键值对的形式dict) '重构。
        该段剧情是整体剧情的{plot_num_frac}部分,plot摘要如下:{plot_str}

        避免使用"读者", "在xx小说中", "在剧情中"，游戏名称等可能打断沉浸感的词汇。
        下面是一个例子：
        {{
            "target":"劳拉的主要目标是荒岛上生存下来，找到并解救她的团队成员，同时探索荒岛。",
            "scene":"劳拉和她的团队在寻找邪马台古国的途中遭遇风暴，船只失事，劳拉漂流到一个未知的岛屿上。这个岛屿是日本以东、福尔摩沙龙三角中的一座小岛——邪马台。那座岛屿以及曾经存在于岛上的王国充满着神秘的传说，许多飞机和船只都曾在附近遭遇狂风暴雨而失事，残骸布满着它的海岸，因而闻名。",
            "role":[
                "劳拉·克劳馥: 幼稚单纯且充满自由精神和荒野情怀。此时劳拉还只是一个21岁的刚踏出校园的小女孩。对亡故的父亲的不解和对冒险的渴望促使劳拉和志同道和的同窗好友珊曼莎一起乘上了驶往邪马台古国的坚忍号，在寻找邪马台古国的途中，劳拉所在的船只遭遇风暴失事，她漂流到一个未知的岛屿上，孤身一人在岛上醒来，被一个野蛮人从背后袭击。醒来后发现自己被吊在一个类似祭台的环境中，周围满是尸骨，开始她的求生之旅。",
                "康拉德·罗斯: 睿智，理性，坚韧而一丝不苟。坚忍号探险船的船长，52岁的资深冒险家，曾在皇家海军陆战队突击旅服役。他与劳拉的父母私交颇深，对队员们的安危非常关心，是劳拉的老师和精神领导者。在风暴中与劳拉一同遭遇船难，之后在岛上与劳拉重逢，共同面对岛上的危险。",
                "珊曼莎·西村: 大大咧咧，习惯过奢侈阔绰的生活。实习纪录片制片人，也是劳拉的同窗好友。她拥有葡萄牙和日本的双重国籍，家境殷实。珊曼莎是太阳女王卑弥呼的后裔，珊曼莎下落不明，但是的背包被劳拉找到，里面有一台DV、火柴和对讲机。",
            ]
        }}
    """,
    input_variables=["novel_name","plot_str","plot_num_frac","story_leading_role","language"],
)
# 并以json格式: 'main:[xx:xx, xx:xx, ...] '重构。


# 3. triggers
# 生成两个triggers, 关键在于不要像过渡
# 意图要明确，字数别太多
# 类似于条件语句：if xxx then xxx
prompt_plot_triggers=PromptTemplate(
    template = """请根据以下情节摘要，描述{novel_name}小说中两个情节之间的过渡条件
        请简洁明了地阐述从当前情节结束并跳转到下一个情节的关键原因，确保原因合理且符合人物行为逻辑
        跳转条件需要与当前情节的主要目标"{plot_target}"以及下一段剧情有所关联，应该是从当前plot变化到下一段plot之间的关键内容
        避免提及"读者", "在xx小说中"等词汇，专注于情节和人物的内在联系。
        如果该情节有其他triggers, 你输出的新剧情触发条件应该与其他跳转条件有较大差别, 以适应不同剧情结果, 请确保不同的跳转条件有明显的差异，并能够合理地引导剧情发展
        
        该段剧情是整体剧情的{plot_num_frac}部分,plot摘要如下:{plot_str}
        下一段plot的摘要如下:{next_episode_str}
        {else_triggers}
        格式示例（请根据实际情况填写**中的内容），用{language}回答：
        <IF> **在当前剧情中需要达成的条件** <THEN> **达成条件后触发的关键事件(事件承上启下，发生于下一段剧情之前)，导致剧情向下一个情节发展**


    """,
    input_variables=["novel_name","plot_str","next_episode_str","plot_target","plot_num_frac","else_triggers","story_leading_role","language"],
)



In [ ]:
# 单线/多线
# 生成具体剧情

if if_single_plot:  # 这个是单线的剧情, 不走上面的多线部分, 但是要重新组织
    temp_dict = single_plot_dict
else:               # 多线剧情，额外的生成内容
    #temp_dict = multi_plot_dict
    pass

plot_gen_list = {k: temp_dict[k] for k in sorted(temp_dict)}
len_main_plot = len(plot_gen_list)

story_dict["Episodes"] = []

i = 0
for key,value in tqdm(plot_gen_list.items()):


    i += 1
    #print(i)
    # 查看固定位置数据
    if i<4:
        #continue
        pass
    if i >5:
        #break
        pass
    
    plot_dict_item = plot_gen_list[key]
    plot_dict = copy.deepcopy(template_dict_plot)
    plot_dict["episode_id"] = key
    plot_dict["pre_episode_id"] = pre_plot_dict[key]
    #plot_dict["episode_desc"] = plot_dict_item["plot_content"]
    plot_num_frac = f"{key}/{len_main_plot+1}"  # 当前情节进程数
    
    plot_dict["episode_desc"] = plot_dict_item["plot_content"]
    
    """
    # 小说抽取的摘要剧情不需要这一部分
    # 1. 情节生成 详细情节 - detail_events
    

    chain_plot_detail = prompt_plot_detail | llm_agent
    res_plot_detail = chain_plot_detail.invoke({"novel_name": novel_name, "plot_str": plot_dict_item["plot_content"],"plot_num_frac":plot_num_frac,"language":language}).content
    #print(res_plot_detail)
    plot_dict["episode_desc"] = res_plot_detail

    #break
    #"""

    # 2. 其他故事信息  - target,scene,role
    chain_plot_info = prompt_plot_info | llm_agent
    res_plot_info = chain_plot_info.invoke({"novel_name": novel_name, "plot_str": plot_dict_item["plot_content"],"plot_num_frac":plot_num_frac,"story_leading_role":story_leading_role,"language":language}).content
    #print("res",res_plot_info)
    try:
        cleaned_plot_string = res_plot_info.split('```json')[1].split('```')[0].strip()
    except:
        cleaned_plot_string = res_plot_info
    try:
        cleaned_plot_string1 = json.loads(cleaned_plot_string)["main"]
    except:
        cleaned_plot_string1 = json.loads(cleaned_plot_string)
    #print(json.dumps(cleaned_plot_string1,indent=2, ensure_ascii=False))   

    
    if type(cleaned_plot_string1) == list:
        plot_info ={}
        for item in cleaned_plot_string1:
            if type(item) == str:
                item = item.replace("：",":")
                item1 = item.split(":",1)
                #print(item1)
                if item1[0] == "role":
                    item2 = item1[1].split(":",1)
                    plot_dict["episode_chars"][item2[0]] = item2[1]
                else:
                    plot_info[item1[0].strip()] = item1[1].strip()
            elif type(item) == dict:
                key1 = list(item.keys())[0]
                plot_info[next(iter(item))] = next(iter(item.values()))
            else:
                print("wrong type!")
                item = item.replace("：",":")

        plot_dict["episode_goal"] = plot_info["target"]
        plot_dict["episode_scene"] = plot_info["scene"]

    elif type(cleaned_plot_string1) == dict:
        #print(cleaned_plot_string1)
        #print( cleaned_plot_string1["target"])
        plot_dict["episode_goal"] = cleaned_plot_string1["target"]
        plot_dict["episode_scene"] = cleaned_plot_string1["scene"]

        for role_item in cleaned_plot_string1["role"]:
            if type(role_item) == str:
                role_item = role_item.replace("：",":")
                item1 = role_item.split(":",1)
                plot_dict["episode_chars"][item1[0]] = item1[1]
            elif type(role_item) == dict:
                plot_dict["episode_chars"][next(iter(role_item))] = next(iter(role_item.values()))
            else:
                print("wrong type!")
                item = item.replace("：",":")
    
        #print(cleaned_plot_string1["target"])
        #print("goal: ",plot_dict["goal"])
    #print("2")
    
    #"""
    # 3. 跳转情节生成 - triggers

    # 需要当前情节的目标
    plot_target = cleaned_plot_string1["target"]

    if plot_dict_item["next_episode"] == []:
        plot_dict["triggers"] = [{}]
    else:
        plot_dict["triggers"] = []
        # 对于每个trigger生成？
        # 但是不确定这样会不会与不同的选项冲突


        # 将生成过的triggers记录一下
        else_triggers = "该plot中没有其他trigger"

        for item in plot_dict_item["next_episode"]:
            #print(item)
            next_key = list(item.keys())[0]
            next_value = item[next_key]

            
            trigger_plot = {"condition":"","next_episode": next_key}
            next_episode_num_frac = f"{next_key}/{len_main_plot+1}"
            

            chain_plot_triggers = prompt_plot_triggers | llm_agent
            #res_plot_triggers = chain_plot_triggers.invoke({"novel_name": novel_name, "plot_str": plot_dict_item["plot_content"], "next_episode_str": next_value, "plot_target" : plot_target,"plot_num_frac":plot_num_frac,"next_episode_num_frac":next_episode_num_frac,"story_leading_role":story_leading_role,"language":language}).content
            res_plot_triggers = chain_plot_triggers.invoke({"novel_name": novel_name, "plot_str": plot_dict_item["plot_content"], "next_episode_str": next_value, "plot_target" : plot_target,"plot_num_frac":plot_num_frac,"else_triggers":else_triggers,"story_leading_role":story_leading_role,"language":language}).content
            
            if else_triggers == "":
                else_triggers = """需要注意的是, 在这段剧情中, 除了你现在需要编写的trigger以外, 还存在其他triggers, 它们可以导致剧情跳转到不同的分支。
                你需要确保你编写的触发条件与其他触发条件有显著差别，以确保剧情能够清晰地沿着不同的路径发展
                具体来说, <IF>之后的条件和<THEN>之后触发的剧情都应与其他triggers的<IF>和<THEN>的内容产生较大的不同
                以下是这个情节中的其他triggers: \n
                """
            else_triggers += res_plot_triggers + "\n"



            #print("当前剧情:\n",plot_dict_item["plot_content"])
            #print(f"过渡剧情:\n{res_plot_triggers}")
            #print(f"下一个剧情:\n{next_value}")
            trigger_plot["condition"] = res_plot_triggers
            plot_dict["triggers"].append(trigger_plot)
    #print("3")
    #break

    #print(json.dumps(plot_dict,indent=2, ensure_ascii=False))  
    story_dict["Episodes"].append(plot_dict)
    
    #break

100%|██████████| 17/17 [08:49<00:00, 31.16s/it]


In [ ]:

#print(cleaned_plot_string)
#print(json.dumps(cleaned_plot_string1,indent=2, ensure_ascii=False)) 
#cleaned_plot_string1 = json.loads(cleaned_plot_string)

In [21]:

print(json.dumps(story_dict["Episodes"],indent=2, ensure_ascii=False))  

[
  {
    "plot_id": 1,
    "pre_plot_id": [
      0
    ],
    "goal": "Harry Potter's main goal is to survive and find safety after the tragic death of his parents, as he is left in the care of the Dursleys.",
    "scene": "The scene is set in Privet Drive, a quiet and ordinary suburban neighborhood where the Dursleys live. The setting is during the night, as Albus Dumbledore and Professor McGonagall discuss the recent events involving Voldemort's defeat and the death of Harry's parents. Hagrid arrives on a flying motorcycle to deliver baby Harry to the Dursleys' doorstep.",
    "plot_des": "CHAPTER ONE THE BOY WHO LIVED\n\nMr. and Mrs. Dursley of Privet Drive, who took pride in their normalcy, were unsettled by strange occurrences and mentions of the Potters, especially their son Harry. Meanwhile, Albus Dumbledore and Professor McGonagall discussed the defeat of Voldemort and the Potters' deaths. Dumbledore decided to leave baby Harry with the Dursleys, believing he would be safer a

In [22]:

prompt_role_list=PromptTemplate(
    template = """请介绍在{novel_name}小说中的出现的主要人物，我会给你一个人物列表（列表中可能会有不重要的角色和重复的角色，需要删减），结合整体剧情详细介绍, 需要依次介绍人物身份、人物性格和人物经历等方面。
        如果其中有重复的人物名称，比如有别名、昵称,不同语言翻译导致的名称不同，或者是只有名字的一部分，这种情况仅保留一个人物名称进行介绍，最后输出的人物数量不大于7。
        输出时按人物重要性排序，先输出在剧情中占比更多的角色。
        
        使用如下格式`**:详细内容`, 保持结果的格式和例子中的格式一致, 并以json格式: 'main:[xx:xx, xx:xx, ...] '重构。
        用{language}回答,不要提到"读者","在xx小说中"，"在剧情中"等令人出戏的字眼，在人物介绍时不要提到游戏名称
        下面是一个例子：
        劳拉·克劳馥: 主角，21岁，刚踏出校园。充满自由精神和荒野情怀，坚韧不拔，作风实际，头脑冷静。在寻找邪马台古国的途中，劳拉所在的船只遭遇风暴失事，她漂流到一个未知的岛屿上，孤身一人在岛上醒来，开始她的求生之旅,在岛上，劳拉学会了生存，经历了从一名经验不足的年轻女孩到一个能够独当一面的冒险家的成长过程。在这个过程中，劳拉不仅要面对凶猛的野生动物和敌对的势力，还要揭开岛上的秘密，寻找失踪的同伴，并最终战胜邪教组织。
        珊曼莎·西村:实习纪录片制片人，也是与劳拉从寄宿学校一直读到伦敦大学的同窗好友。为人大大咧咧，习惯过着奢侈阔绰的生活。拥有葡萄牙和日本的双重国籍，家境殷实。由于她是太阳女王卑弥呼的后裔，所以马蒂亚斯企图将卑弥呼的灵魂植入她体内以复活太阳女王。
        詹姆斯·惠特曼：考古学家，也是当红考古电视节目“惠特曼的世界”主持人。久负盛名但是自私冷漠，对财富和名声有着异呼寻常的渴望，心胸狭窄，为了私利不择手段。坚忍号的这次探险计划其实就是为“惠特曼的世界”下一季做准备。劳拉原来十分崇拜惠特曼博士，但在荒岛遇险过程中惠特曼所表现出的自私冷漠以及背叛使劳拉看清了他的真面目。
        马蒂亚斯:反派boss，太阳兄弟会的首领。神经怪异，但行事却条理清晰，冷酷无情，为了实现自己的目标不惜牺牲他人。1982年，马蒂亚斯驾机迫降在小岛附近海域。在逃亡过程中，他发现了岛上隐藏的邪马台古国遗址，并得知太阳女王拥有不可思议的魔力。31年的孤岛生活使他变得疯狂而且偏执，他采用威胁和恐吓手段笼统了一帮遇难者组成太阳兄弟会，还在女王遗址外修建了一座破烂小城。当他知道坚忍号失事船员里的珊曼莎是卑弥呼后裔后，就开始了丧心病狂的活人献祭阴谋。
        ...
        人物列表如下:{role_list}
    """,
    input_variables=["novel_name","role_list","language"],
)
# 你需要找出其中最重要的至多6个人物进行介绍
# 总人物整合
prompt_role_list_unknown=PromptTemplate(
    template = """请介绍在{novel_name}小说中的出现的主要人物
        我会给你这个故事中每一幕的人物列表（列表中可能会有不重要的角色和重复的角色，需要删减），结合每一幕的人物状态整体介绍, 需要依次介绍人物身份、人物性格和人物经历等方面。
        如果其中有重复的人物名称，比如有别名、昵称,不同语言翻译导致的名称不同，或者是只有名字的一部分，这种情况仅保留一个人物名称进行介绍，最后输出的人物数量不大于7
        需要依次介绍人物身份、人物性格和人物经历
        输出时按人物重要性排序，先输出在剧情中占比更多的角色。

        使用如下格式`**:详细内容`, 保持结果的格式和例子中的格式一致, 并以json格式: 'main:[xx:xx, xx:xx, ...] '重构。
        用{language}回答,不要提到"读者","在xx小说中"，"在剧情中"等令人出戏的字眼，在人物介绍时不要提到游戏名称
        下面是一个输出格式的例子：
        劳拉·克劳馥: 主角，21岁，刚踏出校园。充满自由精神和荒野情怀，坚韧不拔，作风实际，头脑冷静。在寻找邪马台古国的途中，劳拉所在的船只遭遇风暴失事，她漂流到一个未知的岛屿上，孤身一人在岛上醒来，开始她的求生之旅,在岛上，劳拉学会了生存，经历了从一名经验不足的年轻女孩到一个能够独当一面的冒险家的成长过程。在这个过程中，劳拉不仅要面对凶猛的野生动物和敌对的势力，还要揭开岛上的秘密，寻找失踪的同伴，并最终战胜邪教组织。
        珊曼莎·西村:实习纪录片制片人，也是与劳拉从寄宿学校一直读到伦敦大学的同窗好友。为人大大咧咧，习惯过着奢侈阔绰的生活。拥有葡萄牙和日本的双重国籍，家境殷实。由于她是太阳女王卑弥呼的后裔，所以马蒂亚斯企图将卑弥呼的灵魂植入她体内以复活太阳女王。
        詹姆斯·惠特曼：考古学家，也是当红考古电视节目“惠特曼的世界”主持人。久负盛名但是自私冷漠，对财富和名声有着异呼寻常的渴望，心胸狭窄，为了私利不择手段。坚忍号的这次探险计划其实就是为“惠特曼的世界”下一季做准备。劳拉原来十分崇拜惠特曼博士，但在荒岛遇险过程中惠特曼所表现出的自私冷漠以及背叛使劳拉看清了他的真面目。
        马蒂亚斯:反派boss，太阳兄弟会的首领。神经怪异，但行事却条理清晰，冷酷无情，为了实现自己的目标不惜牺牲他人。1982年，马蒂亚斯驾机迫降在小岛附近海域。在逃亡过程中，他发现了岛上隐藏的邪马台古国遗址，并得知太阳女王拥有不可思议的魔力。31年的孤岛生活使他变得疯狂而且偏执，他采用威胁和恐吓手段笼统了一帮遇难者组成太阳兄弟会，还在女王遗址外修建了一座破烂小城。当他知道坚忍号失事船员里的珊曼莎是卑弥呼后裔后，就开始了丧心病狂的活人献祭阴谋。
        ...
        人物列表如下:{role_list}
    """,
    input_variables=["novel_name","role_list","language"],
)

In [ ]:
# 最后整体补齐story所有人物信息
# 单线多线都有


#print(json.dumps(story_dict,indent=2, ensure_ascii=False))  
# 统计一下总人物
role_str = ""  


if_role_known = False
# 第一种是根据人物列表直接总结，认为llm知道所有人物信息

if if_role_known:
    role_str_set = set()  
    for plot in story_dict["Episodes"]:  
        for role in plot["episode_chars"]:  
            if role not in role_str_set: 
                role_str += f"{role}, " 
                role_str_set.add(role)  #
    role_str.strip().strip(",")
    print("role_list: ",role_str)
    chain_role_list = prompt_role_list | llm_agent
    res_role_list = chain_role_list.invoke({"novel_name": novel_name, "role_list": role_str, "language":language}).content
# 第二种更试用于小说llm, llm不了解人物时根据所有人物状态总结
else:
    
    for i in range(len(story_dict["Episodes"])):  
        plot = story_dict["Episodes"][i]
        role_str += f"故事的第{i+1}幕出场的人物以及状态如下:\n"
        for role in plot["episode_chars"]:  
            role_str += role + ": " + plot["episode_chars"][role] + "\n"

    role_str.strip().strip("\n")
    print("role_list: ",role_str)

    chain_role_list = prompt_role_list_unknown | llm_agent
    res_role_list = chain_role_list.invoke({"novel_name": novel_name, "role_list": role_str, "language":language}).content

cleaned_role_string = res_role_list.split('```json')[1].split('```')[0].strip()
role_list_str = json.loads(cleaned_role_string)["main"]
if type(role_list_str) == list:
    for role_item in role_list_str:
        # 多个str组成的list：['伊森·温特斯: 主角...',...]
        if type(role_item) == str:
            role_item = role_item.replace("：",":")
            item1 = role_item.split(":",1)
            story_dict["Settings"]["story_chars"][item1[0]] = item1[1]
        # 多个dict组成的list：[{'伊森·温特斯': '主角...'},...,{}]
        elif type(role_item) == dict:
            for key,value in role_item.items():
                story_dict["Settings"]["story_chars"][key] = value
        else:
            print(type(role_item))
            print("类型错误！")
    
elif type(role_list_str) == dict:
    for key,value in role_list_str.items():
        story_dict["Settings"]["story_chars"][key] = value


print(json.dumps(story_dict["Settings"]["story_chars"],indent=2, ensure_ascii=False))  
#print(json.dumps(story_dict,indent=2, ensure_ascii=False))  

role_list:  故事的第1幕出场的人物以及状态如下:
Harry Potter:  A baby at this point, innocent and unaware of the significant events that have transpired. He is the sole survivor of an attack by Voldemort, earning him the title 'the boy who lived.'
Albus Dumbledore:  Wise, kind, and powerful, Dumbledore is the headmaster of Hogwarts School of Witchcraft and Wizardry. He decides that Harry will be safer growing up away from the fame of the magical world.
Professor McGonagall:  Stern but caring, McGonagall is a professor at Hogwarts. She is concerned about Harry's well-being and initially skeptical about leaving him with the Dursleys.
Hagrid:  Loyal and gentle, Hagrid is a giant who works at Hogwarts. He is entrusted with the task of delivering Harry to the Dursleys, showing his deep affection and protective nature towards Harry.
Mr. and Mrs. Dursley:  Ordinary and proud of their normalcy, they are unsettled by the strange occurrences and the mention of the Potters. They reluctantly become Harry's guardia

In [24]:
#print(cleaned_role_string)
print(json.dumps(story_dict["Settings"],indent=2, ensure_ascii=False)) 

{
  "story_id": "",
  "story_name": "HarryPotter1-HarryPotterAndTheSorcerersStone",
  "story_style": [
    "magical adventure",
    "coming-of-age",
    "whimsical",
    "mysterious",
    "heartwarming",
    "humorous",
    "fantastical"
  ],
  "leading_role": "Harry Potter",
  "story_desc": "Set in the magical world of Hogwarts School of Witchcraft and Wizardry, as well as various locations in the non-magical world, the story follows Harry Potter, an orphan who discovers on his eleventh birthday that he is a wizard. He is introduced to the magical world, where he learns about his parents' legacy, his own magical abilities, and the dark wizard Voldemort who killed his parents. The school is filled with enchanted creatures, magical spells, and hidden secrets, creating an atmosphere of wonder and discovery.",
  "story_target": "The main goal is for Harry to uncover the mystery of the Sorcerer's Stone, a magical object that grants immortality and immense power, which is hidden within Hogw

In [25]:
#print(res_role_list)
game_single_file_path = './data/game_gen_11_23_SM.jsonl'
with jsonlines.open(game_single_file_path, mode='a') as writer:
    writer.write(story_dict)

### 后面是原先写的prompt，可以参考一下

In [ ]:
prompt_summary = PromptTemplate(
            template="""
Rewrite a summary of a novel into a sequential plot tree.
Each plot has a segment of content, connected by key nodes. The plot is a refinement of the original content, but should not be too short. The key nodes should be important events in the upper and lower segments of the plot, and the details of the key nodes and how they make the plot change should be specifically written.
When summarizing, it is necessary to consider the impact of the entire text, such as: a person may have multiple identities, which were not revealed in the previous text, and should be explained in the plot tree
The plot and key points of the plot tree should fully cover all the main content of the original novel. The proportion of each part in the plot tree should conform to the proportion of the original chapter plot, and should not be too general. The important characters repeatedly mentioned in the original abstract should have their names appear in the plot tree.
The plot and key nodes should not exceed 7, which means a maximum of 7 plots, 7 key nodes, and one ending can provide an overview of the entire text.

Output as a string in the following format:
Plot 1:*****
Key Node 1:****
Plot 2:***
....
Conclusion:***

The original summary is as follows{ori_sum}""",
            input_variables=["ori_sum"],
    )
chain = prompt_summary | llm_agent

with open(novel_plot_path, 'r', encoding='utf-8') as file:
    content_plot_sum = file.read()  # 读取文件内容
print(f"char: {len(content_plot_sum)}, token: {len(enc.encode(content_plot_sum))}")

#"""
gpt_plot_tree = chain.invoke(content_plot_sum)
print(gpt_plot_tree.response_metadata["token_usage"])
print(gpt_plot_tree.content)

plot_tree_file_path =  novel_plot_path.replace("_plot.txt","_plot_tree.txt")
with open(plot_tree_file_path, 'w', encoding='utf-8') as f:
    f.write( gpt_plot_tree.content )
#"""
# token 10k 也可以生成

In [ ]:
prompt_tree_multi = PromptTemplate(
            template="""
I have a plot tree for a novel. The original plot tree was a continuous single line plot, and now you need to write some additional plots, key nodes, and endings for these plots to make it a multi branch story.
Key nodes represent several different branches, and implementing different conditions on key nodes can trigger the same or different events. In the end, all choices can jump to the outcome, but may achieve the same or different results.
You need to write down in detail all the different new storylines, key nodes, triggering conditions for node jumps, and endings.
At each key node, there must be at least two options to become a key node (must jump to the different storylines), and it is necessary to explain the specific content of each option, the reason for the jump, and which plot or ending it will jump to. The key nodes must jump to. When a new plot and ending appear, specific descriptions are required. If a certain ending appears, there must be a key node and its branch that jumps to that ending.
You need to consider the clues throughout the text, as a person may have multiple identities in the text. Your new plot also needs to consider whether a certain plot will conflict with the choices made by the previous branch nodes or the plot, and the choices made by other characters should be in line with the characteristics of the corresponding characters. If two branches lead to the same plot, then the appearance of this plot must conform to the plot logic of different branches.
It should be noted that the key plot and original ending (as end1) of the original text must appear, and all endings and plot logic need to be consistent due to the influence of previous plots and choices. However, the differences between different endings should be significant, and the endings cannot be homogenized. The number of different endings must be greater than 2 and less than 6

The final output format is as follows, with ** representing the corresponding specific content, All plots and key nodes should ultimately lead to ending
Plot 1: ** (a certain location/time/event)
Key Node 1: ** (Encountering critical events, need to make a selection)
(Each key node has at least 2 options and at most 4 options)
(option 1) **(Possible choices made in the face of critical nodes and the consequences of those choices), it will jump to Plot *(* refers to a specific corresponding plot, and the reason for the jump should be reasonable)
(option 2) **, it will jump to Plot 4
Plot 2: **
Key Node 2:**
(option 1) **, it will jump to Plot 3
(option 2) **, it will jump to Plot 4
(option 3) **, it will jump to ending 1
Ending 1:**(The events encountered earlier and all the option made ultimately resulted in the Ending)
Ending 2:**(Different Ending should be consistent with the plot and have significant differences)
...

The original plot tree is as follows{ori_plot_tree}""",
            input_variables=["ori_plot_tree"],
    )
chain = prompt_tree_multi | llm_agent

with open(plot_tree_file_path, 'r', encoding='utf-8') as file:
    gpt_plot_tree = file.read()  # 读取文件内容

gpt_plot_tree_multi = chain.invoke(gpt_plot_tree)
print(gpt_plot_tree_multi.response_metadata["token_usage"])
print(gpt_plot_tree_multi.content)

plot_tree_multi_path =  plot_tree_file_path.replace("_plot_tree.txt","_plot_tree_multi.txt")
with open(plot_tree_multi_path, 'w', encoding='utf-8') as f:
    f.write( gpt_plot_tree_multi.content )

In [ ]:
prompt_tree_multi = PromptTemplate(
            template="""
Objective: Create a multi-branch story tree.

Requirements:
Each key node must have at least two options and at most four options.
Each option should describe the possible choices and the consequences of those choices.
All plot points and key moments should ultimately jump to an Ending.
Consider clues throughout the text; a character may have multiple identities.
New plots should consider whether they conflict with choices made by previous branch nodes or the plot.
Choices made by other characters should align with their respective characteristics.
If two branches lead to the same plot, the appearance of this plot must conform to the plot logic of different branches.
The key plot and original ending (as Ending 1) must appear.
All endings and plot logic must be consistent due to the influence of previous plots and choices.
The differences between different endings should be significant; endings cannot be homogenized.
The number of different endings must be greater than 2 and less than 6.

Output Format:
Plot 1: ** (a certain location/time/event)
Key Node 1: ** (Encountering critical events, needing to make a selection)
(Each key node has at least 2 options and at most 4 options)
(Option 1) ** (Choices made in the face of critical nodes and the consequences of those choices), it will jump to Plot *(* refers to a specific corresponding plot, and the reason for the jump should be reasonable)
(Option 2) **, it will jump to Plot 4
Plot 2: **
Key Node 2:**
(Option 1) **, it will jump to Plot 3
(Option 2) **, it will jump to Plot 4
(Option 3) **, it will jump to Ending 1
Ending 1: ** (The events encountered earlier and all the options made ultimately resulted in the Ending, The ending must be reasonable)
Ending 2: ** (There should be significant differences in different Ending)
...


The original plot tree is as follows{ori_plot_tree}""",
            input_variables=["ori_plot_tree"],
    )
chain = prompt_tree_multi | llm_agent

with open(plot_tree_file_path, 'r', encoding='utf-8') as file:
    gpt_plot_tree = file.read()  # 读取文件内容

gpt_plot_tree_multi = chain.invoke(gpt_plot_tree)
print(gpt_plot_tree_multi.response_metadata["token_usage"])
print(gpt_plot_tree_multi.content)

plot_tree_multi_path =  plot_tree_file_path.replace("_plot_tree.txt","_plot_tree_multi.txt")
with open(plot_tree_multi_path, 'w', encoding='utf-8') as f:
    f.write( gpt_plot_tree_multi.content )